#Clone Repository

In [0]:
!rm -rf *
!rm -rf .git* .config
!ls -la

In [0]:
!git clone https://github.com/manuhg/yolo_retrain.git .

# Do it at once

In [0]:
!python train.py -d . -ts NFPA -m yolov2-tiny

 # Do it step by step

## Get  Pascal VOC dataset

In [0]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

!cat 2007_train.txt 2012_*.txt > train.txt

!cp 2007_val.txt test.txt

!cp data/voc.names classes.txt

!mkdir bkup
!mv 2* bkup/
!mv *.tar bkup/

## Get NFPA dataset

In [0]:
!pip install gdown

In [0]:
!gdown https://drive.google.com/uc?id=1_gJcci9p6cS0EucumFt2gpwA0tqgP7_f&export=download

In [0]:
!unzip NFPA_dataset.zip

In [0]:
!mkdir nfpa

In [0]:
!mv 'NFPA dataset' nfpa/data

In [0]:
!cp nfpa/data/train.txt ./
!cp nfpa/data/test.txt ./
!printf "NFPA" > classes.txt

## Train

### Prepare config

In [0]:
data_dir = '.'
class_names_file = 'classes.txt'
batch_size, subdivisions = int('64'), int('8')
model_name='yolov2'
model_url='https://pjreddie.com/media/files/darknet19_448.conv.23'
filename='yolo_custom.cfg'

import os
import sys
import subprocess
from gen_files import gen
import argparse

def exec_cmd(cmdstr):
    print(cmdstr,os.popen(cmdstr).read())

In [0]:
if not os.path.isfile('./darknet'):
    print('Cloning darknet repository')
    exec_cmd('git clone https://github.com/pjreddie/darknet.git dkn')
    exec_cmd('mv -v dkn/* ./')
    exec_cmd("sed -i 's/GPU=0/GPU=1/;s/CUDNN=0/CUDNN=1/;s/OPENCV=0/OPENCV=1/;s/OPENMP=0/OPENMP=1/;' Makefile")
    exec_cmd('make -j8')

if not os.path.isfile('darknet19_448.conv.23'):
    print('Downloading darknet model')
    exec_cmd('wget '+model_url)

In [0]:
with open(data_dir+'/'+class_names_file) as f:
            class_names = list(
                map(lambda s: s.replace('\n', '').strip(), f.readlines()))
            class_names = list(filter(None,class_names))

data_file, names_file, cfg_file = gen(
    class_names, model_name=model_name, batch_size=batch_size, subdivisions=subdivisions, filename=filename)
flag = True
print(data_file, names_file, cfg_file)

###  Train the model

#### sub process

In [0]:
print('Traning the model')
cmd = './darknet detector train '+data_file +' '+cfg_file+' darknet19_448.conv.23'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1,shell = True)
for line in iter(p.stdout.readline,''):
    print(line)
p.stdout.close()
p.wait()

#### Direct commandline

In [0]:
# !./darknet detector train cfg/obj.data cfg/yolo_custom.cfg darknet19_448.conv.23